In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math
import matplotlib.pyplot as plt
import torch.nn.init as init

In [2]:
def train_data(N_x, x_l, x_r, del_x, N_t, t_i, t_f, del_t, N_bc, N_ic):

    x_train = np.geomspace(x_l+0.001, x_r-0.001, N_x)
    x_train = np.tile(x_train, N_t)
    x_bc1 = np.zeros(N_bc)
    x_bc2 = np.ones(N_bc)*x_r
    x_ic = np.random.uniform(x_l, x_r, N_ic)
    x_train = np.concatenate((x_train,x_bc1,x_bc2,x_ic),0)
    x_train = torch.FloatTensor(x_train)
    x_train = x_train.unsqueeze(-1)
    x_train = x_train.clone().detach().requires_grad_(True)

    t_start = t_i + 0.001 
    t_end = t_start + del_t*(N_t-1) 
    t_array = np.geomspace(t_start, t_end, N_t)
    t_train = np.repeat(t_array, N_x)
    t_bc = np.random.uniform(t_i, t_f, 2*N_bc)
    t_ic = np.zeros(N_ic)
    t_train = np.concatenate((t_train, t_bc, t_ic),0)
    t_train = torch.FloatTensor(t_train)
    t_train = t_train.unsqueeze(-1)
    t_train = t_train.clone().detach().requires_grad_(True)
    
    return x_train, t_train

def xavier_init(m):
    if isinstance(m, nn.Linear):
        init.xavier_uniform_(m.weight)
        if m.bias is not None:
            init.constant_(m.bias, 0.0)
    
class FVMANN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FVMANN, self).__init__()
        
        # Fully conected model
        modules = []
        for i in range(len(hidden_size)):
            if(i==0):
                modules.append(nn.Linear(input_size, hidden_size[i]))
            else:
                modules.append(nn.Linear(hidden_size[i-1], hidden_size[i]))  
            modules.append(nn.Tanh())
                
        modules.append(nn.Linear(hidden_size[-1], output_size))
        modules.append(nn.Tanh())
        self.fc = nn.Sequential(*modules)
        self.fc.apply(xavier_init)
        
    def forward(self, x_train, t_train):
        op = self.fc(torch.cat((x_train, t_train),1))
        op_t = torch.autograd.grad(op, t_train, grad_outputs=torch.ones_like(op), create_graph=True)[0]
        op_x = torch.autograd.grad(op, x_train, grad_outputs=torch.ones_like(op), create_graph=True)[0]
        op_x2 = torch.autograd.grad(op_x, x_train, grad_outputs=torch.ones_like(op_x), create_graph=True)[0]
        return op, op_t, op_x2
    
def train_model(model, optimiser, epochs, T_r, T_l, k1, N_x, x_l, x_r, del_x, N_t, t_i, t_f, del_t, N_bc, N_ic, T_ini):
    
    loss_store = []
    mse = nn.MSELoss(reduction='sum')
    model.train()  
    
    w1 = 1
    w2 = 6
    w3 = 6
    w4 = 1
        
    N_tot = N_t*N_x + 2*N_bc + N_ic
    null = torch.zeros(N_tot)
    null = null.unsqueeze(-1)

    for epoch in range(epochs):

        x_train, t_train = train_data(N_x, x_l, x_r, del_x, N_t, t_i, t_f, del_t, N_bc, N_ic)
        T, dTdt, d2Tdx2 = model(x_train, t_train)

        eq1 = w1*mse(dTdt, k1*d2Tdx2)/(N_tot)
        ic1 = w2*mse( torch.mul(torch.where(t_train == t_i,1,0),(T - T_ini )), null )/N_ic
        bc1 = w3*mse( torch.mul(torch.where(x_train == x_l,1,0),(T - T_l)), null )/N_bc
        bc2 = w4*mse( torch.mul(torch.where(x_train == x_r,1,0),(T - T_r)), null )/N_bc

        optimiser.zero_grad()
        loss = eq1 + bc1 + bc2 + ic1
        loss.backward()
        optimiser.step()

        loss_store.append(loss.detach().numpy())

        if epoch%200==0:   
            print('epoch = ',epoch)
            print('loss = ',loss.detach().numpy())
            print('eq1_loss = ',eq1.detach().numpy())
            print('ic1_loss = ',ic1.detach().numpy())
            print('bc1_loss = ',bc1.detach().numpy())
            print('bc2_loss = ',bc2.detach().numpy())

    return loss_store

In [3]:
N_x = 35
N_t = 35
N_bc = 100
N_ic = 100
x_l = 0
x_r = 0.7
t_i = 0
t_f = 0.8
del_t = (t_f - (t_i + 0.001))/(N_t - 1)
del_x = (x_r - x_l - 0.002)/(N_x - 1)
T_l = 1.0
T_r = 0.0
T_ini = 0

# Neural network params
input_size = 2
hidden_size = [45, 45, 45, 45, 45, 45]
output_size = 1

# material params
k1 = 0.05

# Training data and initial data
model = FVMANN(input_size, hidden_size, output_size)
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total trainable parameters in the model:", total_trainable_params)

# Setup Loss function and Optimiser
learning_rate = 6e-6
optimiser = torch.optim.AdamW(model.parameters(), lr=learning_rate)
epochs = 40000

# Training model
loss_store = train_model(model, optimiser, epochs, T_r, T_l, k1, N_x, x_l, x_r, del_x, N_t, t_i, t_f, del_t, N_bc, N_ic, T_ini)

Total trainable parameters in the model: 10531
epoch =  0
loss =  4.231027
eq1_loss =  0.3396477
ic1_loss =  0.029313808
bc1_loss =  3.747417
bc2_loss =  0.11464849
epoch =  200
loss =  3.3788536
eq1_loss =  0.6682879
ic1_loss =  0.017351072
bc1_loss =  2.5542092
bc2_loss =  0.13900542


KeyboardInterrupt: 